<a href="https://colab.research.google.com/github/Hemashree2407/SQL_Project1/blob/main/Equity%20Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance ta --quiet

In [ ]:
# @title Main Script
# Enhanced Indian Equity Analyzer - Fixed Version
# This script includes all bug fixes and enhancements for analyzing Indian stocks

import yfinance as yf
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Tuple
from dataclasses import dataclass
import json
import warnings
warnings.filterwarnings('ignore')

@dataclass
class StockScore:
    symbol: str
    fundamental_score: float
    technical_score: float
    sentiment_score: float
    composite_score: float
    analysis_date: str
    key_metrics: Dict
    data_quality_score: float  # New field to track data completeness

class EquityAnalyzer:
    def __init__(self, db_path: str = "equity_analysis.db"):
        self.db_path = db_path
        self.setup_database()
        self._initialize_industry_benchmarks()

    def setup_database(self):
        """Initialize SQLite database for storing analysis results"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS stock_analysis (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                symbol TEXT,
                analysis_date TEXT,
                fundamental_score REAL,
                technical_score REAL,
                sentiment_score REAL,
                composite_score REAL,
                data_quality_score REAL,
                key_metrics TEXT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        ''')
        conn.commit()
        conn.close()

    def _initialize_industry_benchmarks(self):
        """Initialize comprehensive industry benchmarks for Indian markets"""
        self.INDUSTRY_BENCHMARKS = {
            'technology': {
                'net_profit_margin': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10},
                'roe': {'excellent': 0.25, 'good': 0.18, 'fair': 0.12},
                'roce': {'excellent': 0.28, 'good': 0.20, 'fair': 0.15},
                'ebitda_margin': {'excellent': 0.25, 'good': 0.18, 'fair': 0.10},
                'debt_equity': {'excellent': 0.3, 'good': 0.5, 'fair': 0.8},
                'current_ratio': {'excellent': 2.0, 'good': 1.5, 'fair': 1.2}
            },
            'financial': {
                'return_on_assets': {'excellent': 0.015, 'good': 0.01, 'fair': 0.008},
                'return_on_equity': {'excellent': 0.15, 'good': 0.12, 'fair': 0.08},
                'net_interest_margin': {'excellent': 0.035, 'good': 0.03, 'fair': 0.025},
                'cost_to_income_ratio': {'excellent': 0.45, 'good': 0.55, 'fair': 0.65},
                'capital_adequacy_ratio': {'excellent': 0.15, 'good': 0.12, 'fair': 0.10},
                'gross_npa_ratio': {'excellent': 0.02, 'good': 0.04, 'fair': 0.06}
            },
            'manufacturing': {
                'net_profit_margin': {'excellent': 0.10, 'good': 0.06, 'fair': 0.03},
                'ebitda_margin': {'excellent': 0.15, 'good': 0.10, 'fair': 0.05},
                'roe': {'excellent': 0.18, 'good': 0.12, 'fair': 0.08},
                'roce': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10},
                'inventory_turnover': {'excellent': 8, 'good': 6, 'fair': 4},
                'receivables_turnover': {'excellent': 10, 'good': 7, 'fair': 5},
                'debt_equity': {'excellent': 0.8, 'good': 1.2, 'fair': 1.8},
                'interest_coverage': {'excellent': 10, 'good': 6, 'fair': 4}
            },
            'pharmaceutical': {
                'net_profit_margin': {'excellent': 0.15, 'good': 0.10, 'fair': 0.07},
                'ebitda_margin': {'excellent': 0.22, 'good': 0.18, 'fair': 0.12},
                'roe': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10},
                'roce': {'excellent': 0.25, 'good': 0.18, 'fair': 0.12},
                'r_and_d_to_revenue': {'excellent': 0.08, 'good': 0.05, 'fair': 0.03},
                'debt_equity': {'excellent': 0.5, 'good': 0.8, 'fair': 1.2}
            },
            'fmcg': {
                'net_profit_margin': {'excellent': 0.12, 'good': 0.08, 'fair': 0.05},
                'roe': {'excellent': 0.30, 'good': 0.20, 'fair': 0.15},
                'roce': {'excellent': 0.35, 'good': 0.25, 'fair': 0.18},
                'inventory_turnover': {'excellent': 12, 'good': 8, 'fair': 6},
                'receivables_turnover': {'excellent': 15, 'good': 10, 'fair': 7},
                'working_capital_turnover': {'excellent': 10, 'good': 7, 'fair': 5}
            },
            'energy': {
                'net_profit_margin': {'excellent': 0.10, 'good': 0.07, 'fair': 0.04},
                'roe': {'excellent': 0.15, 'good': 0.10, 'fair': 0.07},
                'roce': {'excellent': 0.12, 'good': 0.08, 'fair': 0.05},
                'debt_equity': {'excellent': 1.0, 'good': 1.5, 'fair': 2.0},
                'interest_coverage': {'excellent': 5, 'good': 3, 'fair': 2}
            },
            'real_estate': {
                'net_profit_margin': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10},
                'roe': {'excellent': 0.15, 'good': 0.10, 'fair': 0.07},
                'debt_equity': {'excellent': 1.5, 'good': 2.0, 'fair': 2.5},
                'current_ratio': {'excellent': 1.5, 'good': 1.2, 'fair': 1.0}
            },
            'telecom': {
                'ebitda_margin': {'excellent': 0.35, 'good': 0.28, 'fair': 0.20},
                'roe': {'excellent': 0.12, 'good': 0.08, 'fair': 0.05},
                'debt_equity': {'excellent': 1.5, 'good': 2.0, 'fair': 2.5},
                'arpu_growth': {'excellent': 0.10, 'good': 0.05, 'fair': 0.02}
            },
            'healthcare': {
                'net_profit_margin': {'excellent': 0.15, 'good': 0.10, 'fair': 0.07},
                'roe': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10},
                'roce': {'excellent': 0.25, 'good': 0.18, 'fair': 0.12},
                'bed_occupancy_rate': {'excellent': 0.80, 'good': 0.70, 'fair': 0.60}
            }
        }

    def get_stock_data(self, symbol: str, period: str = "1y") -> Optional[Dict[str, any]]:
        """Fetch comprehensive stock data from Yahoo Finance with enhanced error handling"""
        try:
            stock = yf.Ticker(symbol)

            # Test if ticker is valid and has market data
            info = stock.info
            if not info or info.get('regularMarketPrice') is None:
                print(f"❌ Invalid ticker or no market data for {symbol}")
                return None

            # Get historical data with retry logic
            try:
                hist_data = stock.history(period=period, timeout=10)
                if hist_data.empty:
                    print(f"❌ No historical data available for {symbol}")
                    return None
            except Exception as hist_e:
                print(f"❌ Error fetching historical data for {symbol}: {str(hist_e)}")
                return None

            # Get financial statements
            balance_sheet = stock.balance_sheet
            income_stmt = stock.financials
            cash_flow = stock.cashflow

            # Get quarterly data for trend analysis
            quarterly_financials = stock.quarterly_financials
            quarterly_balance_sheet = stock.quarterly_balance_sheet

            return {
                'price_data': hist_data,
                'info': info,
                'balance_sheet': balance_sheet,
                'income_statement': income_stmt,
                'cash_flow': cash_flow,
                'quarterly_financials': quarterly_financials,
                'quarterly_balance_sheet': quarterly_balance_sheet
            }
        except Exception as e:
            print(f"❌ Error fetching data for {symbol}: {str(e)}")
            return None

    def classify_industry(self, info: Dict) -> str:
        """Enhanced industry classification for Indian markets"""
        sector = info.get('sector', '').lower()
        industry = info.get('industry', '').lower()

        # Financial services
        if any(keyword in sector or keyword in industry for keyword in
               ['bank', 'financ', 'insurance', 'capital markets', 'nbfc']):
            return 'financial'

        # Technology/IT
        elif any(keyword in sector or keyword in industry for keyword in
                 ['technology', 'software', 'it services', 'internet', 'computer']):
            return 'technology'

        # Manufacturing/Auto
        elif any(keyword in sector or keyword in industry for keyword in
                 ['auto', 'vehicle', 'machinery', 'industrial', 'manufacturing']):
            return 'manufacturing'

        # Pharmaceutical
        elif any(keyword in sector or keyword in industry for keyword in
                 ['pharma', 'drug', 'biotech', 'healthcare products']):
            return 'pharmaceutical'

        # FMCG
        elif any(keyword in sector or keyword in industry for keyword in
                 ['consumer goods', 'fmcg', 'beverages', 'food products', 'personal care']):
            return 'fmcg'

        # Energy
        elif any(keyword in sector or keyword in industry for keyword in
                 ['energy', 'oil', 'gas', 'power', 'utilities', 'coal']):
            return 'energy'

        # Real Estate
        elif any(keyword in sector or keyword in industry for keyword in
                 ['real estate', 'realty', 'construction', 'infrastructure']):
            return 'real_estate'

        # Telecom
        elif any(keyword in sector or keyword in industry for keyword in
                 ['telecom', 'communication', 'mobile services']):
            return 'telecom'

        # Healthcare
        elif any(keyword in sector or keyword in industry for keyword in
                 ['healthcare', 'hospital', 'diagnostic', 'medical']):
            return 'healthcare'

        else:
            return 'general'

    def safe_divide(self, numerator: float, denominator: float, default: float = 0.0) -> float:
        """Enhanced safe division with proper handling of edge cases"""
        try:
            # Check for None, NaN, or invalid inputs
            if pd.isna(numerator) or pd.isna(denominator):
                return default

            # Check for zero denominator
            if denominator == 0:
                return default

            # Perform division
            result = numerator / denominator

            # Check if result is finite
            if not np.isfinite(result):
                return default

            return result
        except Exception:
            return default

    def validate_financial_data(self, stock_data: Dict) -> Tuple[bool, float]:
        """Validate financial data completeness and quality"""
        quality_score = 100.0
        issues = []

        # Check for essential data
        bs = stock_data.get('balance_sheet')
        is_ = stock_data.get('income_statement')
        cf = stock_data.get('cash_flow')

        if bs is None or bs.empty:
            quality_score -= 30
            issues.append("Missing balance sheet data")

        if is_ is None or is_.empty:
            quality_score -= 30
            issues.append("Missing income statement data")

        if cf is None or cf.empty:
            quality_score -= 20
            issues.append("Missing cash flow data")

        # Check for negative values where they shouldn't exist
        info = stock_data.get('info', {})

        if info.get('marketCap', 0) < 0:
            quality_score -= 10
            issues.append("Invalid market cap")

        if info.get('sharesOutstanding', 0) <= 0:
            quality_score -= 10
            issues.append("Invalid shares outstanding")

        # Return validation result
        is_valid = quality_score >= 50  # Minimum threshold for analysis

        if issues:
            print(f"Data quality issues: {', '.join(issues)}")

        return is_valid, quality_score / 100.0

    def _get_value(self, df, possible_keys, default=0, col=0):
        """Enhanced value extraction with better error handling"""
        if df is None or df.empty:
            return default

        # Ensure column index is valid
        if col >= len(df.columns):
            col = 0

        for key in possible_keys:
            if key in df.index:
                try:
                    # Handle series vs dataframe
                    if isinstance(df.loc[key], pd.Series):
                        if col < len(df.loc[key]):
                            value = df.loc[key].iloc[col]
                        else:
                            value = df.loc[key].iloc[0]
                    else:
                        value = df.loc[key]

                    # Validate value
                    if pd.notna(value) and np.isfinite(float(value)):
                        return float(value)
                except (ValueError, TypeError, IndexError, KeyError):
                    continue

        return default

    def calculate_financial_ratios(self, stock_data: Dict) -> Dict:
        """Calculate comprehensive financial ratios with industry-specific adjustments"""
        ratios = {}

        try:
            bs = stock_data.get('balance_sheet')
            is_ = stock_data.get('income_statement')
            cf = stock_data.get('cash_flow')
            info = stock_data.get('info', {})

            if bs is None or bs.empty or is_ is None or is_.empty:
                print("[Ratio Calc] Missing or empty financial statements.")
                return ratios

            # Determine industry
            industry_type = self.classify_industry(info)
            ratios['industry_classification'] = industry_type

            # Get fundamental values
            total_assets = self._get_value(bs, ['Total Assets', 'Total Asset', 'totalAssets'], 0)
            total_liabilities = self._get_value(bs, ['Total Liab', 'Total Liabilities', 'totalLiab'], 0)
            current_assets = self._get_value(bs, ['Total Current Assets', 'Current Assets', 'totalCurrentAssets'], 0)
            current_liabilities = self._get_value(bs, ['Total Current Liabilities', 'Current Liabilities', 'totalCurrentLiabilities'], 0)
            cash = self._get_value(bs, ['Cash', 'Cash And Cash Equivalents', 'cashAndCashEquivalents'], 0)
            inventory = self._get_value(bs, ['Inventory', 'inventory'], 0)
            receivables = self._get_value(bs, ['Net Receivables', 'Receivables', 'netReceivables'], 0)

            # Income Statement items
            revenue = self._get_value(is_, ['Total Revenue', 'Revenue', 'totalRevenue'], 0)
            cogs = self._get_value(is_, ['Cost Of Revenue', 'Cost Of Goods Sold', 'costOfRevenue'], 0)
            operating_income = self._get_value(is_, ['Operating Income', 'operatingIncome'], 0)
            net_income = self._get_value(is_, ['Net Income', 'netIncome'], 0)
            ebit = self._get_value(is_, ['Ebit', 'EBIT', 'ebit'], operating_income)
            interest_expense = abs(self._get_value(is_, ['Interest Expense', 'interestExpense'], 0))

            # Derived values
            shareholders_equity = self.safe_divide(total_assets, 1) - self.safe_divide(total_liabilities, 1) if total_assets > 0 else self._get_value(bs, ['Total Stockholder Equity', 'Stockholders Equity', 'totalStockholderEquity'], 0)
            gross_profit = self.safe_divide(revenue, 1) - self.safe_divide(cogs, 1) if cogs > 0 else self.safe_divide(revenue, 1) * 0.3
            working_capital = self.safe_divide(current_assets, 1) - self.safe_divide(current_liabilities, 1)

            # Calculate general ratios applicable to all industries
            # LIQUIDITY RATIOS
            ratios['current_ratio'] = self.safe_divide(current_assets, current_liabilities)
            ratios['quick_ratio'] = self.safe_divide(current_assets - inventory, current_liabilities)
            ratios['cash_ratio'] = self.safe_divide(cash, current_liabilities)
            ratios['working_capital'] = working_capital

            # LEVERAGE RATIOS
            ratios['debt_equity'] = self.safe_divide(total_liabilities, shareholders_equity)
            ratios['debt_ratio'] = self.safe_divide(total_liabilities, total_assets)
            ratios['equity_ratio'] = self.safe_divide(shareholders_equity, total_assets)
            ratios['interest_coverage'] = self.safe_divide(ebit, interest_expense) if interest_expense > 0 else float('inf') if ebit > 0 else 0

            # PROFITABILITY RATIOS
            ratios['gross_profit_margin'] = self.safe_divide(gross_profit, revenue)
            ratios['operating_profit_margin'] = self.safe_divide(operating_income, revenue)
            ratios['net_profit_margin'] = self.safe_divide(net_income, revenue)
            ratios['roe'] = self.safe_divide(net_income, shareholders_equity)
            ratios['roa'] = self.safe_divide(net_income, total_assets)

            # ROCE calculation
            capital_employed = self.safe_divide(total_assets, 1) - self.safe_divide(current_liabilities, 1)
            ratios['roce'] = self.safe_divide(ebit, capital_employed)

            # EFFICIENCY RATIOS
            ratios['asset_turnover'] = self.safe_divide(revenue, total_assets)
            ratios['inventory_turnover'] = self.safe_divide(cogs, inventory) if inventory > 0 else 0
            ratios['receivables_turnover'] = self.safe_divide(revenue, receivables) if receivables > 0 else 0
            ratios['days_inventory'] = self.safe_divide(365, ratios.get('inventory_turnover', 0)) if ratios.get('inventory_turnover', 0) > 0 else 0
            ratios['days_receivables'] = self.safe_divide(365, ratios.get('receivables_turnover', 0)) if ratios.get('receivables_turnover', 0) > 0 else 0

            # MARKET RATIOS
            market_cap = info.get('marketCap', 0)
            shares_outstanding = info.get('sharesOutstanding', info.get('impliedSharesOutstanding', 1))
            current_price = info.get('currentPrice', info.get('regularMarketPrice', 0))

            ratios['pe_ratio'] = info.get('trailingPE', info.get('forwardPE', 0))
            ratios['peg_ratio'] = info.get('pegRatio', 0)
            ratios['pb_ratio'] = self.safe_divide(market_cap, shareholders_equity) if shareholders_equity > 0 and market_cap > 0 else info.get('priceToBook', 0)
            ratios['ps_ratio'] = self.safe_divide(market_cap, revenue) if revenue > 0 and market_cap > 0 else 0
            ratios['earnings_per_share'] = info.get('trailingEps', self.safe_divide(net_income, shares_outstanding))

            # FIXED: Book value per share calculation
            ratios['book_value_per_share'] = self.safe_divide(shareholders_equity, shares_outstanding) if shares_outstanding > 0 else 0

            # FREE CASH FLOW ANALYSIS
            if cf is not None and not cf.empty:
                operating_cash_flow = self._get_value(cf, ['Total Cash From Operating Activities', 'Operating Cash Flow', 'totalCashFromOperatingActivities'], 0)
                capex = abs(self._get_value(cf, ['Capital Expenditures', 'capitalExpenditures'], 0))
                free_cash_flow = operating_cash_flow - capex

                ratios['free_cash_flow'] = free_cash_flow
                ratios['fcf_margin'] = self.safe_divide(free_cash_flow, revenue)
                ratios['fcf_per_share'] = self.safe_divide(free_cash_flow, shares_outstanding) if shares_outstanding > 0 else 0

            # GROWTH METRICS
            if is_ is not None and len(is_.columns) >= 2:
                current_revenue = self._get_value(is_, ['Total Revenue', 'Revenue', 'totalRevenue'], 0, col=0)
                previous_revenue = self._get_value(is_, ['Total Revenue', 'Revenue', 'totalRevenue'], 0, col=1)

                ratios['revenue_growth_yoy'] = self.safe_divide(current_revenue - previous_revenue, previous_revenue)

                current_earnings = self._get_value(is_, ['Net Income', 'netIncome'], 0, col=0)
                previous_earnings = self._get_value(is_, ['Net Income', 'netIncome'], 0, col=1)

                ratios['earnings_growth_yoy'] = self.safe_divide(current_earnings - previous_earnings, previous_earnings)
            else:
                ratios['revenue_growth_yoy'] = info.get('revenueGrowth', 0)
                ratios['earnings_growth_yoy'] = info.get('earningsGrowth', 0)

            # Add industry-specific ratios
            self._add_industry_specific_ratios(ratios, stock_data, industry_type)

        except Exception as e:
            print(f"[Ratio Calc] Error during calculation: {e}")

        # Ensure no NaN or Inf values remain
        for key, value in ratios.items():
            if isinstance(value, (int, float)) and (not pd.notna(value) or np.isinf(value)):
                ratios[key] = 0

        return ratios

    def _add_industry_specific_ratios(self, ratios: Dict, stock_data: Dict, industry_type: str):
        """Add industry-specific ratios based on classification"""
        bs = stock_data.get('balance_sheet')
        is_ = stock_data.get('income_statement')
        info = stock_data.get('info', {})

        if industry_type == 'financial':
            # Banking specific ratios
            net_interest_income = self._get_value(is_, ['Net Interest Income', 'netInterestIncome'], 0)
            net_loans = self._get_value(bs, ['Net Loans', 'netLoans'], 0)
            deposits = self._get_value(bs, ['Deposits', 'deposits'], 0)

            ratios['net_interest_margin'] = self.safe_divide(net_interest_income, net_loans) if net_loans > 0 else 0
            ratios['loan_deposit_ratio'] = self.safe_divide(net_loans, deposits) if deposits > 0 else 0

        elif industry_type == 'pharmaceutical':
            # Pharma specific ratios
            r_and_d_expense = self._get_value(is_, ['Research Development', 'researchAndDevelopment'], 0)
            revenue = ratios.get('revenue', self._get_value(is_, ['Total Revenue', 'Revenue', 'totalRevenue'], 0))

            ratios['r_and_d_to_revenue'] = self.safe_divide(r_and_d_expense, revenue) if revenue > 0 else 0

        elif industry_type == 'fmcg':
            # FMCG specific ratios
            revenue = ratios.get('revenue', self._get_value(is_, ['Total Revenue', 'Revenue', 'totalRevenue'], 0))
            working_capital = ratios.get('working_capital', 0)

            ratios['working_capital_turnover'] = self.safe_divide(revenue, working_capital) if working_capital > 0 else 0

    def calculate_risk_return_metrics(self, symbol: str, price_data: pd.DataFrame) -> Dict[str, float]:
        """Calculate risk and return metrics with NSE/BSE specific adjustments"""
        if price_data.empty or len(price_data) < 60:
            return self._get_default_risk_metrics()

        try:
            # Create a copy to avoid modifying original data
            price_data = price_data.copy()

            # Calculate daily returns
            price_data['Return'] = price_data['Close'].pct_change()

            # Remove any infinite or NaN values
            price_data['Return'] = price_data['Return'].replace([np.inf, -np.inf], np.nan)
            price_data = price_data.dropna(subset=['Return'])

            if len(price_data) < 20:
                return self._get_default_risk_metrics()

            # RETURN METRICS
            avg_return_daily = price_data['Return'].mean()
            avg_return_annual = avg_return_daily * 252  # Indian markets ~252 trading days

            # Handle geometric mean calculation properly
            returns_plus_one = 1 + price_data['Return']
            if (returns_plus_one > 0).all():
                geometric_mean_return = (returns_plus_one.prod()) ** (1/len(price_data['Return'])) - 1
            else:
                geometric_mean_return = avg_return_daily

            cumulative_return = (price_data['Close'].iloc[-1] / price_data['Close'].iloc[0]) - 1 if price_data['Close'].iloc[0] != 0 else 0

            # RISK METRICS
            volatility_daily = price_data['Return'].std()
            volatility_annual = volatility_daily * np.sqrt(252)
            variance = price_data['Return'].var()

            # Downside risk
            negative_returns = price_data['Return'][price_data['Return'] < 0]
            downside_deviation = negative_returns.std() if len(negative_returns) > 1 else 0

            # Value at Risk
            var_95 = np.percentile(price_data['Return'], 5) if len(price_data['Return']) > 0 else 0

            # Maximum Drawdown
            rolling_max = price_data['Close'].expanding().max()
            drawdown = (price_data['Close'] - rolling_max) / rolling_max
            max_drawdown = drawdown.min()

            # MARKET CORRELATION AND BETA (Using NIFTY 50 as benchmark)
            beta, correlation, alpha_annual = self._calculate_beta_correlation(price_data, symbol)

            # Risk-adjusted returns
            risk_free_rate = 0.065  # Indian 10-year G-Sec yield approximation
            sharpe_ratio = self.safe_divide(avg_return_annual - risk_free_rate, volatility_annual)
            sortino_ratio = self.safe_divide(avg_return_annual - risk_free_rate, downside_deviation * np.sqrt(252)) if downside_deviation > 0 else 0

            return {
                'average_return_daily': avg_return_daily,
                'average_return_annual': avg_return_annual,
                'geometric_mean_return': geometric_mean_return,
                'cumulative_return': cumulative_return,
                'volatility_daily': volatility_daily,
                'volatility_annual': volatility_annual,
                'variance': variance,
                'downside_deviation': downside_deviation,
                'value_at_risk_95': var_95,
                'max_drawdown': max_drawdown,
                'beta': beta,
                'correlation_with_index': correlation,
                'alpha': alpha_annual,
                'sharpe_ratio': sharpe_ratio,
                'sortino_ratio': sortino_ratio,
                'information_ratio': 0  # Placeholder for future implementation
            }

        except Exception as e:
            print(f"[Risk/Return] Error for {symbol}: {str(e)}")
            return self._get_default_risk_metrics()

    def _calculate_beta_correlation(self, price_data: pd.DataFrame, symbol: str) -> Tuple[float, float, float]:
        """Calculate beta and correlation with NIFTY 50"""
        try:
            # Use NIFTY 50 as benchmark
            index_symbol = "^NSEI"

            start_date = price_data.index[0]
            end_date = price_data.index[-1]

            # Remove timezone if present
            if hasattr(start_date, 'tz'):
                start_date = start_date.tz_localize(None)
            if hasattr(end_date, 'tz'):
                end_date = end_date.tz_localize(None)

            # Fetch index data
            index_data = yf.download(index_symbol, start=start_date, end=end_date, progress=False)['Close']

            if len(index_data) > 0:
                index_return = index_data.pct_change().dropna()

                # Align dates
                combined = pd.concat([price_data['Return'], index_return], axis=1, join='inner')
                combined.columns = ['stock_return', 'market_return']
                combined = combined.dropna()

                if len(combined) > 20:
                    # Beta calculation
                    covariance = combined.cov().iloc[0, 1]
                    market_variance = combined['market_return'].var()
                    beta = self.safe_divide(covariance, market_variance, default=1)

                    # Correlation
                    correlation = combined.corr().iloc[0, 1]

                    # Alpha calculation
                    risk_free_rate = 0.065 / 252  # Daily risk-free rate
                    excess_stock_return = combined['stock_return'] - risk_free_rate
                    excess_market_return = combined['market_return'] - risk_free_rate

                    alpha = excess_stock_return.mean() - beta * excess_market_return.mean()
                    alpha_annual = alpha * 252

                    return beta, correlation, alpha_annual

            return 1.0, 0.0, 0.0

        except Exception as e:
            print(f"[Beta Calc] Error: {str(e)}")
            return 1.0, 0.0, 0.0

    def _get_default_risk_metrics(self) -> Dict[str, float]:
        """Return default risk metrics when calculation fails"""
        return {
            'average_return_daily': 0.0,
            'average_return_annual': 0.0,
            'geometric_mean_return': 0.0,
            'cumulative_return': 0.0,
            'volatility_daily': 0.0,
            'volatility_annual': 0.0,
            'variance': 0.0,
            'downside_deviation': 0.0,
            'value_at_risk_95': 0.0,
            'max_drawdown': 0.0,
            'beta': 1.0,
            'correlation_with_index': 0.0,
            'alpha': 0.0,
            'sharpe_ratio': 0.0,
            'sortino_ratio': 0.0,
            'information_ratio': 0.0
        }

    def calculate_technical_indicators(self, price_data: pd.DataFrame) -> Dict[str, float]:
        """Calculate technical indicators with circuit breaker awareness"""
        if price_data is None or price_data.empty or len(price_data) < 20:
            return {}

        try:
            indicators = {}
            current_price = price_data['Close'].iloc[-1]

            # Moving Averages
            if len(price_data) >= 20:
                price_data['SMA_20'] = price_data['Close'].rolling(window=20).mean()
                price_data['EMA_20'] = price_data['Close'].ewm(span=20, adjust=False).mean()
                indicators['price_vs_sma20'] = self.safe_divide(current_price - price_data['SMA_20'].iloc[-1], price_data['SMA_20'].iloc[-1])

            if len(price_data) >= 50:
                price_data['SMA_50'] = price_data['Close'].rolling(window=50).mean()
                indicators['price_vs_sma50'] = self.safe_divide(current_price - price_data['SMA_50'].iloc[-1], price_data['SMA_50'].iloc[-1])

            if len(price_data) >= 200:
                price_data['SMA_200'] = price_data['Close'].rolling(window=200).mean()
                indicators['price_vs_sma200'] = self.safe_divide(current_price - price_data['SMA_200'].iloc[-1], price_data['SMA_200'].iloc[-1])

            # RSI
            if len(price_data) >= 14:
                delta = price_data['Close'].diff()
                gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
                loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
                rs = self.safe_divide(gain.iloc[-1], loss.iloc[-1])
                indicators['rsi'] = 100 - self.safe_divide(100, (1 + rs))
            else:
                indicators['rsi'] = 50

            # MACD
            if len(price_data) >= 26:
                exp1 = price_data['Close'].ewm(span=12, adjust=False).mean()
                exp2 = price_data['Close'].ewm(span=26, adjust=False).mean()
                macd = exp1 - exp2
                signal = macd.ewm(span=9, adjust=False).mean()
                indicators['macd'] = macd.iloc[-1]
                indicators['macd_signal'] = signal.iloc[-1]
                indicators['macd_histogram'] = macd.iloc[-1] - signal.iloc[-1]

            # Bollinger Bands
            if len(price_data) >= 20:
                bb_sma = price_data['Close'].rolling(window=20).mean()
                bb_std = price_data['Close'].rolling(window=20).std()
                bb_upper = bb_sma + (bb_std * 2)
                bb_lower = bb_sma - (bb_std * 2)
                indicators['bb_upper'] = bb_upper.iloc[-1]
                indicators['bb_lower'] = bb_lower.iloc[-1]
                band_width = bb_upper.iloc[-1] - bb_lower.iloc[-1]
                indicators['bb_position'] = self.safe_divide(current_price - bb_lower.iloc[-1], band_width, default=0.5)

            # Volume indicators
            if 'Volume' in price_data.columns and len(price_data) >= 20:
                volume_sma = price_data['Volume'].rolling(window=20).mean()
                indicators['volume_sma_ratio'] = self.safe_divide(price_data['Volume'].iloc[-1], volume_sma.iloc[-1], default=1)

            # Price momentum
            if len(price_data) >= 11:
                indicators['momentum_10d'] = self.safe_divide(current_price, price_data['Close'].iloc[-11]) - 1
            if len(price_data) >= 31:
                indicators['momentum_30d'] = self.safe_divide(current_price, price_data['Close'].iloc[-31]) - 1

            # Support and Resistance
            if len(price_data) >= 20:
                indicators['resistance_1m'] = price_data['High'].iloc[-20:].max()
                indicators['support_1m'] = price_data['Low'].iloc[-20:].min()

            # Circuit breaker levels (NSE/BSE specific)
            indicators['upper_circuit'] = current_price * 1.20  # 20% upper circuit
            indicators['lower_circuit'] = current_price * 0.80  # 20% lower circuit

            # Delivery percentage (if available)
            if 'Deliverable Volume' in price_data.columns:
                delivery_pct = self.safe_divide(price_data['Deliverable Volume'].iloc[-1], price_data['Volume'].iloc[-1]) * 100
                indicators['delivery_percentage'] = delivery_pct

            return indicators

        except Exception as e:
            print(f"[Technical] Error: {str(e)}")
            return {}

    def analyze_fundamentals(self, stock_data: Dict, ratios: Dict) -> float:
        """Score fundamental analysis with industry-specific benchmarks"""
        score = 0.0
        info = stock_data.get('info', {})
        industry_type = ratios.get('industry_classification', self.classify_industry(info))
        benchmarks = self.INDUSTRY_BENCHMARKS.get(industry_type, self.INDUSTRY_BENCHMARKS.get('general', {}))

        # Adjust weights based on industry
        if industry_type == 'financial':
            weights = {
                'profitability': 35,
                'asset_quality': 25,
                'liquidity': 10,
                'leverage': 20,
                'growth': 10
            }
        elif industry_type in ['technology', 'pharmaceutical']:
            weights = {
                'profitability': 30,
                'liquidity': 15,
                'efficiency': 20,
                'leverage': 15,
                'growth': 20
            }
        elif industry_type == 'fmcg':
            weights = {
                'profitability': 25,
                'liquidity': 15,
                'efficiency': 30,
                'leverage': 15,
                'growth': 15
            }
        else:
            weights = {
                'profitability': 30,
                'liquidity': 20,
                'efficiency': 20,
                'leverage': 15,
                'growth': 15
            }

        # Score based on benchmarks
        score += self._score_profitability(ratios, benchmarks, industry_type) * weights.get('profitability', 30) / 100
        score += self._score_liquidity(ratios, benchmarks, industry_type) * weights.get('liquidity', 20) / 100
        score += self._score_efficiency(ratios, benchmarks, industry_type) * weights.get('efficiency', 20) / 100
        score += self._score_leverage(ratios, benchmarks, industry_type) * weights.get('leverage', 15) / 100
        score += self._score_growth(ratios, benchmarks, industry_type) * weights.get('growth', 15) / 100

        # Add asset quality score for financial companies
        if industry_type == 'financial' and 'asset_quality' in weights:
            score += self._score_asset_quality(ratios, benchmarks) * weights['asset_quality'] / 100

        return min(score, 100.0)

    def _score_ratio(self, value: float, benchmark: Dict, higher_is_better: bool = True) -> float:
        """Score a single ratio against benchmarks"""
        if not benchmark:
            return 50.0  # Default score if no benchmark

        excellent = benchmark.get('excellent', 0)
        good = benchmark.get('good', 0)
        fair = benchmark.get('fair', 0)

        if higher_is_better:
            if value >= excellent:
                return 100.0
            elif value >= good:
                return 75.0
            elif value >= fair:
                return 50.0
            else:
                return 25.0
        else:  # Lower is better
            if value <= excellent:
                return 100.0
            elif value <= good:
                return 75.0
            elif value <= fair:
                return 50.0
            else:
                return 25.0

    def _score_profitability(self, ratios: Dict, benchmarks: Dict, industry_type: str) -> float:
        """Score profitability metrics"""
        score = 0.0
        count = 0

        # Net Profit Margin
        if 'net_profit_margin' in benchmarks:
            score += self._score_ratio(ratios.get('net_profit_margin', 0), benchmarks['net_profit_margin'])
            count += 1

        # ROE
        if 'roe' in benchmarks:
            score += self._score_ratio(ratios.get('roe', 0), benchmarks['roe'])
            count += 1

        # ROCE
        if 'roce' in benchmarks:
            score += self._score_ratio(ratios.get('roce', 0), benchmarks['roce'])
            count += 1

        # EBITDA Margin (if applicable)
        if 'ebitda_margin' in benchmarks:
            ebitda_margin = ratios.get('operating_profit_margin', 0) * 1.2  # Approximation
            score += self._score_ratio(ebitda_margin, benchmarks['ebitda_margin'])
            count += 1

        return score / count if count > 0 else 50.0

    def _score_liquidity(self, ratios: Dict, benchmarks: Dict, industry_type: str) -> float:
        """Score liquidity metrics"""
        if industry_type == 'financial':
            # Financial companies have different liquidity requirements
            return 75.0  # Default good score for financial companies

        score = 0.0

        # Current Ratio
        current_ratio = ratios.get('current_ratio', 0)
        if current_ratio > 2.0:
            score += 50.0
        elif current_ratio > 1.5:
            score += 40.0
        elif current_ratio > 1.0:
            score += 30.0
        else:
            score += 10.0

        # Quick Ratio
        quick_ratio = ratios.get('quick_ratio', 0)
        if quick_ratio > 1.5:
            score += 50.0
        elif quick_ratio > 1.0:
            score += 40.0
        elif quick_ratio > 0.75:
            score += 30.0
        else:
            score += 10.0

        return score

    def _score_efficiency(self, ratios: Dict, benchmarks: Dict, industry_type: str) -> float:
        """Score efficiency metrics"""
        score = 0.0
        count = 0

        # Asset Turnover
        if ratios.get('asset_turnover', 0) > 0:
            if industry_type == 'financial':
                score += 75.0 if ratios['asset_turnover'] > 0.05 else 50.0
            else:
                score += 75.0 if ratios['asset_turnover'] > 1.0 else 50.0
            count += 1

        # Inventory Turnover (not for financial/service companies)
        if industry_type not in ['financial', 'technology'] and 'inventory_turnover' in benchmarks:
            score += self._score_ratio(ratios.get('inventory_turnover', 0), benchmarks['inventory_turnover'])
            count += 1

        # Receivables Turnover
        if 'receivables_turnover' in benchmarks:
            score += self._score_ratio(ratios.get('receivables_turnover', 0), benchmarks['receivables_turnover'])
            count += 1

        return score / count if count > 0 else 50.0

    def _score_leverage(self, ratios: Dict, benchmarks: Dict, industry_type: str) -> float:
        """Score leverage metrics"""
        score = 0.0

        # Debt to Equity
        if 'debt_equity' in benchmarks:
            score += self._score_ratio(ratios.get('debt_equity', 0), benchmarks['debt_equity'], higher_is_better=False) * 0.5
        else:
            # Default scoring
            de_ratio = ratios.get('debt_equity', 0)
            if de_ratio < 0.5:
                score += 50.0
            elif de_ratio < 1.0:
                score += 40.0
            elif de_ratio < 2.0:
                score += 25.0
            else:
                score += 10.0

        # Interest Coverage
        if 'interest_coverage' in benchmarks:
            score += self._score_ratio(ratios.get('interest_coverage', 0), benchmarks['interest_coverage']) * 0.5
        else:
            ic_ratio = ratios.get('interest_coverage', 0)
            if ic_ratio > 5:
                score += 50.0
            elif ic_ratio > 3:
                score += 40.0
            elif ic_ratio > 1.5:
                score += 25.0
            else:
                score += 10.0

        return score

    def _score_growth(self, ratios: Dict, benchmarks: Dict, industry_type: str) -> float:
        """Score growth metrics"""
        score = 0.0

        # Revenue Growth
        revenue_growth = ratios.get('revenue_growth_yoy', 0)
        if revenue_growth > 0.20:
            score += 50.0
        elif revenue_growth > 0.15:
            score += 40.0
        elif revenue_growth > 0.10:
            score += 30.0
        elif revenue_growth > 0.05:
            score += 20.0
        else:
            score += 10.0

        # Earnings Growth
        earnings_growth = ratios.get('earnings_growth_yoy', 0)
        if earnings_growth > 0.20:
            score += 50.0
        elif earnings_growth > 0.15:
            score += 40.0
        elif earnings_growth > 0.10:
            score += 30.0
        elif earnings_growth > 0.05:
            score += 20.0
        else:
            score += 10.0

        return score

    def _score_asset_quality(self, ratios: Dict, benchmarks: Dict) -> float:
        """Score asset quality for financial companies"""
        score = 75.0  # Default score

        # Gross NPA Ratio (if available)
        if 'gross_npa_ratio' in benchmarks and 'gross_npa_ratio' in ratios:
            score = self._score_ratio(ratios['gross_npa_ratio'], benchmarks['gross_npa_ratio'], higher_is_better=False)

        return score

    def analyze_technicals(self, price_data: pd.DataFrame, indicators: Dict) -> float:
        """Score technical analysis"""
        score = 0.0

        try:
            # RSI scoring
            rsi = indicators.get('rsi', 50)
            if 30 <= rsi <= 70:
                score += 20.0
            elif rsi < 30:
                score += 25.0  # Oversold
            elif rsi > 70:
                score += 5.0   # Overbought

            # Moving average trend
            if indicators.get('price_vs_sma20', 0) > 0:
                score += 15.0
            if indicators.get('price_vs_sma50', 0) > 0:
                score += 10.0
            if indicators.get('price_vs_sma200', 0) > 0:
                score += 10.0

            # MACD
            if indicators.get('macd_histogram', 0) > 0:
                score += 15.0

            # Bollinger Bands
            bb_pos = indicators.get('bb_position', 0.5)
            if 0.2 <= bb_pos <= 0.8:
                score += 10.0
            elif bb_pos < 0.2:
                score += 15.0  # Near lower band

            # Volume
            if indicators.get('volume_sma_ratio', 1) > 1.2:
                score += 10.0

            # Momentum
            if indicators.get('momentum_30d', 0) > 0:
                score += 10.0

            # Delivery percentage (NSE specific)
            if indicators.get('delivery_percentage', 0) > 60:
                score += 5.0

            return min(score, 100.0)

        except Exception as e:
            print(f"[Technical Scoring] Error: {str(e)}")
            return 50.0

    def analyze_sentiment(self, stock_data: Dict) -> float:
        """Enhanced sentiment scoring"""
        score = 50.0  # Default neutral

        try:
            info = stock_data.get('info', {})

            # Analyst recommendations
            recommendation = info.get('recommendationKey', '')
            recommendation_map = {
                'strong_buy': 90.0,
                'buy': 75.0,
                'hold': 50.0,
                'sell': 25.0,
                'strong_sell': 10.0
            }

            if recommendation in recommendation_map:
                score = recommendation_map[recommendation]

            # Adjust based on analyst count
            analyst_count = info.get('numberOfAnalystOpinions', 0)
            if analyst_count > 10:
                score *= 1.1
            elif analyst_count > 5:
                score *= 1.05
            elif analyst_count < 3 and analyst_count > 0:
                score *= 0.95

            # Target price vs current price
            target_price = info.get('targetMeanPrice', 0)
            current_price = info.get('currentPrice', 0)

            if target_price > 0 and current_price > 0:
                upside = (target_price - current_price) / current_price
                if upside > 0.20:
                    score = min(score + 10, 100)
                elif upside > 0.10:
                    score = min(score + 5, 100)
                elif upside < -0.10:
                    score = max(score - 10, 0)

            return min(max(score, 0), 100)

        except Exception as e:
            print(f"[Sentiment Scoring] Error: {str(e)}")
            return 50.0

    def analyze_stock(self, symbol: str) -> Optional[StockScore]:
        """Perform comprehensive analysis on a single stock"""
        print(f"\n📊 Analyzing {symbol}...")

        # Fetch data
        stock_data = self.get_stock_data(symbol)
        if not stock_data:
            return None

        # Validate data
        is_valid, data_quality = self.validate_financial_data(stock_data)
        if not is_valid:
            print(f"⚠️ Data quality too low for {symbol} (score: {data_quality:.1%})")
            return None

        # Calculate all metrics
        financial_ratios = self.calculate_financial_ratios(stock_data)
        risk_return_metrics = self.calculate_risk_return_metrics(symbol, stock_data['price_data'])
        technical_indicators = self.calculate_technical_indicators(stock_data['price_data'])

        # Score each component
        fundamental_score = float(self.analyze_fundamentals(stock_data, financial_ratios))
        technical_score = float(self.analyze_technicals(stock_data['price_data'], technical_indicators))
        sentiment_score = float(self.analyze_sentiment(stock_data))

        # Adjust composite score based on data quality
        composite_score = (fundamental_score * 0.4 +
                         technical_score * 0.35 +
                         sentiment_score * 0.25) * data_quality

        # Compile key metrics
        key_metrics = {
            'price': stock_data['info'].get('currentPrice', 0),
            'sector': stock_data['info'].get('sector', 'Unknown'),
            'industry': stock_data['info'].get('industry', 'Unknown'),
            'financial_ratios': financial_ratios,
            'risk_return': risk_return_metrics,
            'technical_indicators': technical_indicators,
            'market_cap': stock_data['info'].get('marketCap', 0),
            '52_week_high': stock_data['info'].get('fiftyTwoWeekHigh', 0),
            '52_week_low': stock_data['info'].get('fiftyTwoWeekLow', 0)
        }

        # Create score object
        score_obj = StockScore(
            symbol=symbol,
            fundamental_score=round(fundamental_score, 1),
            technical_score=round(technical_score, 1),
            sentiment_score=round(sentiment_score, 1),
            composite_score=round(composite_score, 1),
            analysis_date=datetime.now().strftime('%Y-%m-%d'),
            key_metrics=key_metrics,
            data_quality_score=round(data_quality, 2)
        )

        # Save to database
        self.save_analysis(score_obj)

        return score_obj

    def save_analysis(self, score: StockScore):
        """Save analysis results to database"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        cursor.execute('''
            INSERT INTO stock_analysis
            (symbol, analysis_date, fundamental_score, technical_score,
             sentiment_score, composite_score, data_quality_score, key_metrics)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            score.symbol,
            score.analysis_date,
            score.fundamental_score,
            score.technical_score,
            score.sentiment_score,
            score.composite_score,
            score.data_quality_score,
            json.dumps(score.key_metrics)
        ))

        conn.commit()
        conn.close()

    def generate_detailed_report(self, score: StockScore) -> str:
        """Generate comprehensive analysis report with fixed display errors"""
        metrics = score.key_metrics
        ratios = metrics.get('financial_ratios', {})
        risk = metrics.get('risk_return', {})
        tech = metrics.get('technical_indicators', {})
        industry_type = ratios.get('industry_classification', 'general')

        report = f"""
{'='*60}
COMPREHENSIVE EQUITY ANALYSIS REPORT
{'='*60}
Symbol: {score.symbol}
Analysis Date: {score.analysis_date}
Current Price: ₹{metrics.get('price', 0):.2f}
Sector: {metrics.get('sector', 'N/A')}
Industry: {metrics.get('industry', 'N/A')}
Industry Classification: {industry_type.upper()}
Data Quality Score: {score.data_quality_score:.1%}

{'='*60}
SCORING SUMMARY
{'='*60}
Fundamental Score: {score.fundamental_score}/100
Technical Score: {score.technical_score}/100
Sentiment Score: {score.sentiment_score}/100
📊 COMPOSITE SCORE: {score.composite_score}/100

{'='*60}
1. ACCOUNTING ANALYTICS - FINANCIAL STATEMENT ANALYSIS
{'='*60}
"""

        # Add industry-specific sections
        if industry_type == 'financial':
            report += self._generate_financial_section(ratios)
        elif industry_type == 'technology':
            report += self._generate_technology_section(ratios)
        elif industry_type == 'manufacturing':
            report += self._generate_manufacturing_section(ratios)
        elif industry_type == 'pharmaceutical':
            report += self._generate_pharma_section(ratios)
        elif industry_type == 'fmcg':
            report += self._generate_fmcg_section(ratios)
        else:
            report += self._generate_general_section(ratios)

        # Add common sections
        report += f"""
VALUATION RATIOS:
  P/E Ratio: {ratios.get('pe_ratio', 0):.2f}
  P/B Ratio: {ratios.get('pb_ratio', 0):.2f}
  P/S Ratio: {ratios.get('ps_ratio', 0):.2f}
  PEG Ratio: {ratios.get('peg_ratio', 0):.2f}
  EPS: ₹{ratios.get('earnings_per_share', 0):.2f}
  Book Value per Share: ₹{ratios.get('book_value_per_share', 0):.2f}

FREE CASH FLOW ANALYSIS:
  Free Cash Flow: ₹{ratios.get('free_cash_flow', 0)/10000000:.2f} Cr
  FCF Margin: {ratios.get('fcf_margin', 0)*100:.1f}%
  FCF per Share: ₹{ratios.get('fcf_per_share', 0):.2f}

GROWTH METRICS:
  Revenue Growth (YoY): {ratios.get('revenue_growth_yoy', 0)*100:.1f}%
  Earnings Growth (YoY): {ratios.get('earnings_growth_yoy', 0)*100:.1f}%

{'='*60}
2. SECURITY MARKET ANALYTICS - RISK & RETURN ANALYSIS
{'='*60}

RETURN METRICS:
  Daily Average Return: {risk.get('average_return_daily', 0)*100:.3f}%
  Annualized Return: {risk.get('average_return_annual', 0)*100:.1f}%
  Cumulative Return (1Y): {risk.get('cumulative_return', 0)*100:.1f}%
  Geometric Mean Return: {risk.get('geometric_mean_return', 0)*100:.3f}%

RISK METRICS:
  Daily Volatility: {risk.get('volatility_daily', 0)*100:.2f}%
  Annual Volatility: {risk.get('volatility_annual', 0)*100:.1f}%
  Downside Deviation: {risk.get('downside_deviation', 0)*100:.2f}%
  Value at Risk (95%): {risk.get('value_at_risk_95', 0)*100:.2f}%
  Maximum Drawdown: {risk.get('max_drawdown', 0)*100:.1f}%

MARKET CORRELATION & SYSTEMATIC RISK:
  Beta: {risk.get('beta', 0):.2f}
  Correlation with NIFTY 50: {risk.get('correlation_with_index', 0):.2f}
  Alpha (Annual): {risk.get('alpha', 0)*100:.1f}%

RISK-ADJUSTED RETURNS:
  Sharpe Ratio: {risk.get('sharpe_ratio', 0):.2f}
  Sortino Ratio: {risk.get('sortino_ratio', 0):.2f}
  Information Ratio: {risk.get('information_ratio', 0):.2f}

{'='*60}
3. TECHNICAL ANALYSIS
{'='*60}

TREND INDICATORS:
  Price vs 20-SMA: {tech.get('price_vs_sma20', 0)*100:.1f}%
  Price vs 50-SMA: {tech.get('price_vs_sma50', 0)*100:.1f}%
  Price vs 200-SMA: {tech.get('price_vs_sma200', 0)*100:.1f}%

MOMENTUM INDICATORS:
  RSI (14): {tech.get('rsi', 0):.1f}
  10-Day Momentum: {tech.get('momentum_10d', 0)*100:.1f}%
  30-Day Momentum: {tech.get('momentum_30d', 0)*100:.1f}%

MACD ANALYSIS:
  MACD Line: {tech.get('macd', 0):.2f}
  Signal Line: {tech.get('macd_signal', 0):.2f}
  MACD Histogram: {tech.get('macd_histogram', 0):.2f}

BOLLINGER BANDS:
  Upper Band: ₹{tech.get('bb_upper', 0):.2f}
  Lower Band: ₹{tech.get('bb_lower', 0):.2f}
  Band Position: {tech.get('bb_position', 0)*100:.1f}%

VOLUME ANALYSIS:
  Volume vs 20-Day Avg: {tech.get('volume_sma_ratio', 0):.2f}x
  Delivery Percentage: {tech.get('delivery_percentage', 0):.1f}%

SUPPORT & RESISTANCE:
  1-Month Resistance: ₹{tech.get('resistance_1m', 0):.2f}
  1-Month Support: ₹{tech.get('support_1m', 0):.2f}
  Upper Circuit: ₹{tech.get('upper_circuit', 0):.2f}
  Lower Circuit: ₹{tech.get('lower_circuit', 0):.2f}

{'='*60}
INVESTMENT RECOMMENDATION
{'='*60}
"""

        # Add recommendation based on scores
        if score.composite_score >= 70:
            recommendation = "STRONG BUY - Excellent fundamentals with positive technical momentum"
        elif score.composite_score >= 60:
            recommendation = "BUY - Good investment opportunity with favorable risk-reward"
        elif score.composite_score >= 50:
            recommendation = "HOLD - Neutral outlook, monitor for better entry points"
        elif score.composite_score >= 40:
            recommendation = "CAUTION - Weak performance indicators, consider alternatives"
        else:
            recommendation = "AVOID - Poor fundamentals and unfavorable technicals"

        report += f"\n{recommendation}\n"

        # Add data quality warning if needed
        if score.data_quality_score < 0.8:
            report += f"\n⚠️ Note: Data quality score is {score.data_quality_score:.1%}. Some metrics may be missing or estimated.\n"

        report += f"\n{'='*60}\n"

        return report

    def _generate_financial_section(self, ratios: Dict) -> str:
        """Generate financial industry specific section"""
        return f"""
BANKING/FINANCIAL SPECIFIC RATIOS:
  Net Interest Margin: {ratios.get('net_interest_margin', 0)*100:.2f}%
  Return on Assets (ROA): {ratios.get('roa', 0)*100:.2f}%
  Return on Equity (ROE): {ratios.get('roe', 0)*100:.2f}%
  Cost-to-Income Ratio: {ratios.get('cost_to_income_ratio', 0):.2f}
  Loan-to-Deposit Ratio: {ratios.get('loan_deposit_ratio', 0):.2f}
  Asset Quality (NPA Ratio): {ratios.get('gross_npa_ratio', 0)*100:.2f}%
  Capital Adequacy Ratio: {ratios.get('capital_adequacy_ratio', 0):.2f}

"""

    def _generate_technology_section(self, ratios: Dict) -> str:
        """Generate technology industry specific section"""
        return f"""
INFORMATION TECHNOLOGY SPECIFIC RATIOS:
  Net Profit Margin: {ratios.get('net_profit_margin', 0)*100:.1f}%
  Operating Margin: {ratios.get('operating_profit_margin', 0)*100:.1f}%
  Return on Equity (ROE): {ratios.get('roe', 0)*100:.1f}%
  Return on Capital Employed (ROCE): {ratios.get('roce', 0)*100:.1f}%
  Current Ratio: {ratios.get('current_ratio', 0):.2f}
  Debt-to-Equity: {ratios.get('debt_equity', 0):.2f}

"""

    def _generate_manufacturing_section(self, ratios: Dict) -> str:
        """Generate manufacturing industry specific section"""
        return f"""
MANUFACTURING/AUTOMOBILE SPECIFIC RATIOS:
  Net Profit Margin: {ratios.get('net_profit_margin', 0)*100:.1f}%
  EBITDA Margin: {ratios.get('operating_profit_margin', 0)*120:.1f}%
  Return on Equity (ROE): {ratios.get('roe', 0)*100:.1f}%
  Return on Capital Employed (ROCE): {ratios.get('roce', 0)*100:.1f}%
  Inventory Turnover: {ratios.get('inventory_turnover', 0):.2f}x
  Receivables Turnover: {ratios.get('receivables_turnover', 0):.2f}x
  Debt-to-Equity: {ratios.get('debt_equity', 0):.2f}
  Interest Coverage: {ratios.get('interest_coverage', 0):.2f}x
  Current Ratio: {ratios.get('current_ratio', 0):.2f}
  Asset Turnover: {ratios.get('asset_turnover', 0):.2f}x

"""

    def _generate_pharma_section(self, ratios: Dict) -> str:
        """Generate pharmaceutical industry specific section"""
        return f"""
PHARMACEUTICAL SPECIFIC RATIOS:
  Net Profit Margin: {ratios.get('net_profit_margin', 0)*100:.1f}%
  EBITDA Margin: {ratios.get('operating_profit_margin', 0)*120:.1f}%
  Return on Equity (ROE): {ratios.get('roe', 0)*100:.1f}%
  Return on Capital Employed (ROCE): {ratios.get('roce', 0)*100:.1f}%
  R&D to Revenue: {ratios.get('r_and_d_to_revenue', 0)*100:.1f}%
  Current Ratio: {ratios.get('current_ratio', 0):.2f}
  Debt-to-Equity: {ratios.get('debt_equity', 0):.2f}
  Asset Turnover: {ratios.get('asset_turnover', 0):.2f}x

"""

    def _generate_fmcg_section(self, ratios: Dict) -> str:
        """Generate FMCG industry specific section"""
        return f"""
FMCG SPECIFIC RATIOS:
  Net Profit Margin: {ratios.get('net_profit_margin', 0)*100:.1f}%
  Gross Profit Margin: {ratios.get('gross_profit_margin', 0)*100:.1f}%
  Return on Equity (ROE): {ratios.get('roe', 0)*100:.1f}%
  Return on Capital Employed (ROCE): {ratios.get('roce', 0)*100:.1f}%
  Inventory Turnover: {ratios.get('inventory_turnover', 0):.2f}x
  Receivables Turnover: {ratios.get('receivables_turnover', 0):.2f}x
  Working Capital Turnover: {ratios.get('working_capital_turnover', 0):.2f}x
  Asset Turnover: {ratios.get('asset_turnover', 0):.2f}x
  Current Ratio: {ratios.get('current_ratio', 0):.2f}

"""

    def _generate_general_section(self, ratios: Dict) -> str:
        """Generate general industry section"""
        return f"""
LIQUIDITY RATIOS:
  Current Ratio: {ratios.get('current_ratio', 0):.2f}
  Quick Ratio: {ratios.get('quick_ratio', 0):.2f}
  Cash Ratio: {ratios.get('cash_ratio', 0):.2f}
  Working Capital: ₹{ratios.get('working_capital', 0)/10000000:.2f} Cr

LEVERAGE/SOLVENCY RATIOS:
  Debt-to-Equity: {ratios.get('debt_equity', 0):.2f}
  Debt Ratio: {ratios.get('debt_ratio', 0):.2f}
  Equity Ratio: {ratios.get('equity_ratio', 0):.2f}
  Interest Coverage: {ratios.get('interest_coverage', 0):.2f}x

PROFITABILITY RATIOS:
  Gross Profit Margin: {ratios.get('gross_profit_margin', 0)*100:.1f}%
  Operating Profit Margin: {ratios.get('operating_profit_margin', 0)*100:.1f}%
  Net Profit Margin: {ratios.get('net_profit_margin', 0)*100:.1f}%
  Return on Equity (ROE): {ratios.get('roe', 0)*100:.1f}%
  Return on Assets (ROA): {ratios.get('roa', 0)*100:.1f}%
  Return on Capital Employed (ROCE): {ratios.get('roce', 0)*100:.1f}%

EFFICIENCY/TURNOVER RATIOS:
  Asset Turnover: {ratios.get('asset_turnover', 0):.2f}x
  Inventory Turnover: {ratios.get('inventory_turnover', 0):.2f}x
  Receivables Turnover: {ratios.get('receivables_turnover', 0):.2f}x
  Days Inventory Outstanding: {ratios.get('days_inventory', 0):.0f} days
  Days Sales Outstanding: {ratios.get('days_receivables', 0):.0f} days

"""

    def compare_stocks(self, symbols: List[str]) -> pd.DataFrame:
        """Compare multiple stocks and return comparison DataFrame"""
        comparison_data = []

        for symbol in symbols:
            score = self.analyze_stock(symbol)
            if score:
                ratios = score.key_metrics.get('financial_ratios', {})
                risk = score.key_metrics.get('risk_return', {})

                comparison_data.append({
                    'Symbol': symbol,
                    'Composite Score': score.composite_score,
                    'Fundamental Score': score.fundamental_score,
                    'Technical Score': score.technical_score,
                    'Industry': ratios.get('industry_classification', 'general'),
                    'P/E Ratio': ratios.get('pe_ratio', 0),
                    'ROE': ratios.get('roe', 0) * 100,
                    'Debt/Equity': ratios.get('debt_equity', 0),
                    'Annual Return': risk.get('average_return_annual', 0) * 100,
                    'Volatility': risk.get('volatility_annual', 0) * 100,
                    'Sharpe Ratio': risk.get('sharpe_ratio', 0),
                    'Beta': risk.get('beta', 0)
                })

        return pd.DataFrame(comparison_data)

    def screen_stocks(self, symbols: List[str], min_score: float = 60.0) -> List[StockScore]:
        """Screen stocks based on minimum composite score"""
        qualified_stocks = []

        for symbol in symbols:
            score = self.analyze_stock(symbol)
            if score and score.composite_score >= min_score:
                qualified_stocks.append(score)

        # Sort by composite score
        qualified_stocks.sort(key=lambda x: x.composite_score, reverse=True)

        return qualified_stocks

# Example usage functions
def test_analyzer():
    """Test the analyzer with sample stocks"""
    analyzer = EquityAnalyzer()

    # Test with different sectors
    test_symbols = [
        'HDFCBANK.NS',   # Financial
        'INFY.NS',       # Technology
        'TATAMOTORS.NS', # Manufacturing
        'SUNPHARMA.NS',  # Pharmaceutical
        'HINDUNILVR.NS', # FMCG
        'RELIANCE.NS'    # Energy
    ]

    for symbol in test_symbols:
        print(f"\n{'='*60}")
        print(f"Testing {symbol}")
        print('='*60)

        score = analyzer.analyze_stock(symbol)
        if score:
            report = analyzer.generate_detailed_report(score)
            print(report)
        else:
            print(f"Failed to analyze {symbol}")

def compare_stocks_demo():
    """Demo stock comparison functionality"""
    analyzer = EquityAnalyzer()

    # Compare stocks
    symbols = ['HDFCBANK.NS', 'ICICIBANK.NS', 'SBIN.NS', 'AXISBANK.NS']
    comparison_df = analyzer.compare_stocks(symbols)

    print("\nStock Comparison:")
    print(comparison_df.to_string())

    # Screen stocks
    qualified = analyzer.screen_stocks(symbols, min_score=60.0)
    print(f"\nQualified stocks (score >= 60):")
    for stock in qualified:
        print(f"{stock.symbol}: {stock.composite_score}")

if __name__ == "__main__":
    # Run test
    test_analyzer()
    # compare_stocks_demo()

In [ ]:
# @title Data Visualization
import sqlite3
import pandas as pd

def get_analysis_results(db_path: str = "equity_analysis.db") -> pd.DataFrame:
    """Retrieves all analysis results from the database and returns as a DataFrame."""
    conn = sqlite3.connect(db_path)
    query = "SELECT symbol, analysis_date, fundamental_score, technical_score, sentiment_score, composite_score, data_quality_score, key_metrics FROM stock_analysis"
    df = pd.read_sql_query(query, conn)
    conn.close()

    # Optionally, parse the key_metrics JSON to include specific metrics in columns
    # This can make the table more useful for direct Excel export
    parsed_metrics = []
    for index, row in df.iterrows():
        metrics = json.loads(row['key_metrics'])
        parsed_metrics.append({
            'Price': metrics.get('price', 0),
            'Sector': metrics.get('sector', 'N/A'),
            'Industry': metrics.get('industry', 'N/A'),
            'PE Ratio': metrics.get('financial_ratios', {}).get('pe_ratio', 0),
            'ROE (%)': metrics.get('financial_ratios', {}).get('roe', 0) * 100,
            'Debt/Equity': metrics.get('financial_ratios', {}).get('debt_equity', 0),
            'Annual Return (%)': metrics.get('risk_return', {}).get('average_return_annual', 0) * 100,
            'Volatility (%)': metrics.get('risk_return', {}).get('volatility_annual', 0) * 100,
            'Sharpe Ratio': metrics.get('risk_return', {}).get('sharpe_ratio', 0),
            'Beta': metrics.get('risk_return', {}).get('beta', 0),
            'RSI': metrics.get('technical_indicators', {}).get('rsi', 0),
            '52 Week High': metrics.get('52_week_high', 0),
            '52 Week Low': metrics.get('52_week_low', 0)
        })

    metrics_df = pd.DataFrame(parsed_metrics)

    # Combine the original DataFrame with the parsed metrics
    # Drop the original key_metrics column if you've parsed everything you need
    df = df.drop(columns=['key_metrics'])
    df = pd.concat([df, metrics_df], axis=1)

    return df

# Retrieve and display the results
analysis_df = get_analysis_results()
display(analysis_df)

In [ ]:
# @title Results table google sheets
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=analysis_df)

In [34]:
# @title Usage
# Initialize analyzer
analyzer = EquityAnalyzer()

# Analyze a single stock (RELIANCE.NS - keeping the original example)
# print("\n" + "="*60)
# print("Analyzing a single stock (RELIANCE.NS)")
# print("="*60)
# score_reliance = analyzer.analyze_stock('RELIANCE.NS')
# if score_reliance:
#     report_reliance = analyzer.generate_detailed_report(score_reliance)
#     print(report_reliance)

# Analyze the new stock (IEX.NS)
print("\n" + "="*60)
print("Analyzing a single stock (IEX.NS)")
print("="*60)
score_iex = analyzer.analyze_stock('IEX.NS')
if score_iex:
    report_iex = analyzer.generate_detailed_report(score_iex)
    print(report_iex)


# Compare multiple stocks (HDFCBANK.NS, ICICIBANK.NS, SBIN.NS - keeping the original example)
# print("\n" + "="*60)
# print("Comparing multiple stocks (HDFCBANK.NS, ICICIBANK.NS, SBIN.NS)")
# print("="*60)
# comparison_df_banks = analyzer.compare_stocks(['HDFCBANK.NS', 'ICICIBANK.NS', 'SBIN.NS'])
# print("\nStock Comparison Summary Table (Banks):")
# display(comparison_df_banks) # Display the summary table

# print("\nGenerating detailed reports for compared bank stocks:")
# for symbol in comparison_df_banks['Symbol'].tolist():
#     print(f"\n{'='*60}")
#     print(f"Detailed Report for {symbol}")
#     print('='*60)
#     # Re-analyze to get the full score object needed for detailed report
#     # Note: This re-fetches data. For efficiency, you might analyze once and store results if analyzing many stocks.
#     score = analyzer.analyze_stock(symbol)
#     if score:
#         report = analyzer.generate_detailed_report(score)
#         print(report)
#     else:
#         print(f"Failed to generate detailed report for {symbol}")

# Compare IEX with competitors based on the user's specified list
print("\n" + "="*60)
print("Comparing IEX.NS with specified competitors")
print("="*60)
# Using the provided list of competitors. Note that Yahoo Finance symbols might vary.
# I'll use readily available symbols for demonstration.
# Direct Exchange Competitors: PXIL (not typically listed as a stock), HPX (not typically listed as a stock), ENEChain (new platform)
# Power Traders / Participants: PTC India Ltd (PTC.NS), Tata Power Trading Company (part of TATAPOWER.NS),
# Adani Electricity (part of ADANIENSOL.NS or ADANITRANS.NS), NVVN (not listed),
# Manikaran Power Ltd (not listed), JSW Energy Trading Ltd (part of JSWENERGY.NS),
# Refex Energy Ltd (part of REFEX.NS)
# I will use the main listed company symbols where possible for comparison from the user's list.
competition_symbols = ['IEX.NS', 'PTC.NS', 'TATAPOWER.NS', 'ADANIENSOL.NS', 'JSWENERGY.NS', 'REFEX.NS']
comparison_df_energy = analyzer.compare_stocks(competition_symbols)
print("\nStock Comparison Summary Table (Specified Competitors):")
display(comparison_df_energy)

print("\nGenerating detailed reports for specified competitor stocks:")
for symbol in comparison_df_energy['Symbol'].tolist():
    print(f"\n{'='*60}")
    print(f"Detailed Report for {symbol}")
    print('='*60)
    # Re-analyze to get the full score object needed for detailed report
    score = analyzer.analyze_stock(symbol)
    if score:
        report = analyzer.generate_detailed_report(score)
        print(report)
    else:
        print(f"Failed to generate detailed report for {symbol}")


# Screen stocks with minimum score (INFY.NS, TCS.NS, WIPRO.NS - keeping the original example)
# print("\n" + "="*60)
# print("Screening stocks (INFY.NS, TCS.NS, WIPRO.NS) with min score >= 60.0")
# print("="*60)
# qualified_stocks = analyzer.screen_stocks(['INFY.NS', 'TCS.NS', 'WIPRO.NS'], min_score=60.0)

# print(f"\nQualified stocks (score >= 60):")
# if qualified_stocks:
#     for stock in qualified_stocks:
#         print(f"{stock.symbol}: {stock.composite_score}")
#         print(f"\n{'='*60}")
#         print(f"Detailed Report for {stock.symbol}")
#         print('='*60)
#         report = analyzer.generate_detailed_report(stock)
#         print(report)
# else:
#     print("No stocks qualified based on the minimum score.")


Analyzing a single stock (IEX.NS)

📊 Analyzing IEX.NS...

COMPREHENSIVE EQUITY ANALYSIS REPORT
Symbol: IEX.NS
Analysis Date: 2025-07-04
Current Price: ₹197.63
Sector: Financial Services
Industry: Capital Markets
Industry Classification: FINANCIAL
Data Quality Score: 100.0%

SCORING SUMMARY
Fundamental Score: 87.8/100
Technical Score: 90.0/100
Sentiment Score: 82.5/100
📊 COMPOSITE SCORE: 87.2/100

1. ACCOUNTING ANALYTICS - FINANCIAL STATEMENT ANALYSIS

BANKING/FINANCIAL SPECIFIC RATIOS:
  Net Interest Margin: 0.00%
  Return on Assets (ROA): 19.54%
  Return on Equity (ROE): 19.54%
  Cost-to-Income Ratio: 0.00
  Loan-to-Deposit Ratio: 0.00
  Asset Quality (NPA Ratio): 0.00%
  Capital Adequacy Ratio: 0.00


VALUATION RATIOS:
  P/E Ratio: 40.92
  P/B Ratio: 8.00
  P/S Ratio: 32.71
  PEG Ratio: 0.00
  EPS: ₹4.83
  Book Value per Share: ₹24.70

FREE CASH FLOW ANALYSIS:
  Free Cash Flow: ₹427.25 Cr
  FCF Margin: 79.5%
  FCF per Share: ₹4.80

GROWTH METRICS:
  Revenue Growth (YoY): 19.6%
  Ear

,Symbol,Composite Score,Fundamental Score,Technical Score,Industry,P/E Ratio,ROE,Debt/Equity,Annual Return,Volatility,Sharpe Ratio,Beta
0,IEX.NS,87.2,87.8,90.0,financial,40.917187,19.537356,0.0,15.292742,35.170238,0.250005,1.0
1,PTC.NS,70.3,74.6,80.0,energy,6.821469,6.970102,0.0,-4.420400,36.219780,-0.301504,1.0
2,TATAPOWER.NS,70.6,46.2,90.0,energy,32.338710,2.533959,0.0,-3.515223,29.769495,-0.336426,1.0
3,ADANIENSOL.NS,68.5,54.9,90.0,energy,99.230770,1.433165,0.0,-1.479187,54.033551,-0.147671,1.0
4,JSWENERGY.NS,61.3,54.9,50.0,energy,45.796062,2.169126,0.0,-26.952836,39.733530,-0.841930,1.0
5,REFEX.NS,61.2,82.5,45.0,energy,35.529934,88.609055,0.0,90.776076,55.492198,1.518701,1.0



Generating detailed reports for specified competitor stocks:

Detailed Report for IEX.NS

📊 Analyzing IEX.NS...

COMPREHENSIVE EQUITY ANALYSIS REPORT
Symbol: IEX.NS
Analysis Date: 2025-07-04
Current Price: ₹197.63
Sector: Financial Services
Industry: Capital Markets
Industry Classification: FINANCIAL
Data Quality Score: 100.0%

SCORING SUMMARY
Fundamental Score: 87.8/100
Technical Score: 90.0/100
Sentiment Score: 82.5/100
📊 COMPOSITE SCORE: 87.2/100

1. ACCOUNTING ANALYTICS - FINANCIAL STATEMENT ANALYSIS

BANKING/FINANCIAL SPECIFIC RATIOS:
  Net Interest Margin: 0.00%
  Return on Assets (ROA): 19.54%
  Return on Equity (ROE): 19.54%
  Cost-to-Income Ratio: 0.00
  Loan-to-Deposit Ratio: 0.00
  Asset Quality (NPA Ratio): 0.00%
  Capital Adequacy Ratio: 0.00


VALUATION RATIOS:
  P/E Ratio: 40.92
  P/B Ratio: 8.00
  P/S Ratio: 32.71
  PEG Ratio: 0.00
  EPS: ₹4.83
  Book Value per Share: ₹24.70

FREE CASH FLOW ANALYSIS:
  Free Cash Flow: ₹427.25 Cr
  FCF Margin: 79.5%
  FCF per Share: ₹4.

In [28]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=comparison_df_energy)

https://docs.google.com/spreadsheets/d/1Lkxyt8s1vKdHi33mkMkzaRC79AG90oCHwa9Q4LZMwIk/edit#gid=0
